# Data Cleaning

In [1]:
import pandas as pd
import numpy as np

import matplotlib as plt
%matplotlib inline

In [2]:
sf_raw = pd.read_csv('sf_raw.csv')

In [3]:
sf_raw.head()

,date,title,link,price,brs,sqft,hood,bath,amenities
0,Oct 1,Lovely Newly Renovated 2BR 1BA Apartment (suns...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2500,2.0,NaN,sunset / parkside,1Ba,"['apartment', 'laundry on site', 'no smoking']"
1,Oct 1,Boo Spooky Special! The Martin,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2680,1.0,591.0,potrero hill,1.5Ba,"['cats are OK - purrr', 'dogs are OK - wooof',..."
2,Oct 1,Arterra condo with Salesforce Tower Skyline Vi...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2800,1.0,NaN,SOMA / south beach,1Ba,"['cats are OK - purrr', 'dogs are OK - wooof',..."
3,Oct 1,Bright Marina Studios,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2100,NaN,NaN,marina / cow hollow,1Ba,['application fee details: 20.00 Credit Check ...
4,Oct 1,Newly Remodeled ! 2 br1ba$3195,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,3195,NaN,NaN,USF / panhandle,1Ba,"['apartment', 'no smoking']"


In [4]:
sf_raw.tail()

,date,title,link,price,brs,sqft,hood,bath,amenities
3099,Sep 30,"$1,895 - Beautiful Studio with Bay Windows and...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1895,NaN,400.0,lower nob hill,1Ba,"['cats are OK - purrr', 'apartment', 'laundry ..."
3100,Sep 30,$2000 Studio/All utilities included/Cat ok,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2000,1.0,NaN,SOMA / south beach,1Ba,"['application fee details: $40.00', 'cats are ..."
3101,Sep 30,$2600 Russian Hill 1 Bedroom/HDWD Flrs/New Kit...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,2600,1.0,NaN,russian hill,1Ba,"['application fee details: $40.00', 'cats are ..."
3102,Sep 30,Pet friendly Courtyard Studio Available Near U...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,1650,NaN,NaN,lower nob hill,NaN,['application fee details: There is a $30 back...
3103,Sep 30,Mission Dolores Park/3809 20th&Dolores/W/D in ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,3495,1.0,NaN,mission district,1Ba,"['application fee details: $35', 'cats are OK ..."


In [5]:
len(sf_raw)

3104

# TO DO

- [ ] BRs NaN -->> 0
- [ ] Bath --> remove 'Ba'
- [ ] hood - create some groupings
- [ ] amenities - create some groupings
- [ ] Deal with SQFT NaNs
